In [19]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
import requests
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub 
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
search_tool = DuckDuckGoSearchRun()

result = search_tool.invoke("PSL Match NEWS today")
result

"PZ vs IU: Where to watch live streaming of today's PSL match? A Sports and PTV Sports will broadcast the match live in Pakistan, while live streaming will be available on the following platforms: Walee Technologies, Tapmad, Tamasha, Daraz, Myco and Begin.. PSL matches are not being live streamed in India currently. You can find the complete live streaming details for PSL 2025 here. Islamabad United won the toss and opted to bat first against Peshawar Zalmi in the 22nd match of the PSL 10 in Lahore on Friday. Close 30.9 C ... ARY NEWS brings you 24/7 Live Streaming, Headlines ... Lahore Qalandars face Islamabad United in the 19th match of the Pakistan Super League 2025 at the Gaddafi Stadium, Lahore tonight. Both teams are enjoying a successful season so far. ISL skipper Shadab Khan won the toss and elected to bowl first on the night. Shadab Khan-led Islamabad United have ... Islamabad United captain Shadab Khan and Peshawar Zalmi skipper Babar Azam at toss ceremony during their match 

In [ ]:
llm = ChatGroq(model="llama3-8b-8192")


In [14]:
llm.invoke("Hi! How are you?")


AIMessage(content="I'm just a language model, I don't have emotions or feelings like humans do, but I'm functioning properly and ready to help you with any questions or tasks you may have! How about you?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 16, 'total_tokens': 58, 'completion_time': 0.035, 'prompt_time': 0.003186116, 'queue_time': 0.409631035, 'total_time': 0.038186116}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run-ea349e5e-6173-45af-94ec-3bc48b03db77-0', usage_metadata={'input_tokens': 16, 'output_tokens': 42, 'total_tokens': 58})

In [21]:
# step 2: Pull the React prompt from LangChain Hub
prompt = hub.pull("hwchase17/react") # Reasoning + action


In [ ]:
# step 3: Create the ReAct agent manually with the pulled prompt

agent = create_react_agent(
    llm = llm,
    tools = [search_tool],
    prompt = prompt
)

In [ ]:
# step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent = agent,
    tools = [search_tool],
    verbose = True
)

In [24]:
# step 5: invoke
response = agent_executor.invoke({'input': "What is the latest news about PSL Match today?"})



> Entering new AgentExecutor chain...
Thought: Let's try to find the latest news about PSL Match today.

Action: duckduckgo_search
Action Input: "latest news about PSL Match today"Peshawar Zalmi will play Islamabad United in the 22nd match of the 2025 Pakistan Super League - Follow today's PZ vs IU PSL 2025 match here, ... Stay up to date with the latest cricket news, player updates, team standings, match highlights, video analysis and live match odds. Related tags: Trending Stories Series Cover Stories. Lahore Qalandars face Islamabad United in the 19th match of the Pakistan Super League 2025 at the Gaddafi Stadium, Lahore tonight. Both teams are enjoying a successful season so far. ISL skipper Shadab Khan won the toss and elected to bowl first on the night. Shadab Khan-led Islamabad United have ... LAHORE: Islamabad United decided to bat first after winning toss against Peshawar Zalmi in the 22nd match of the Pakistan Super League 10 at the Gaddafi Stadium on Friday.Peshawar would 

In [25]:
print(response)

{'input': 'What is the latest news about PSL Match today?', 'output': 'The latest news about PSL Match today is that Peshawar Zalmi will play Islamabad United in the 22nd match of the 2025 Pakistan Super League.'}
